In [ ]:
import gradio as gr
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn

In [10]:
import os
import pandas as pd

### first we need to import our libraries dataset and convert them into .csv

In [6]:
spam_folder = "enron2/spam"
ham_folder = "enron2/ham"

In [7]:
def load_emails(folder, label):
    emails = []
    for filename in os.listdir(folder):
        if filename.endswith(".txt"):
            with open(os.path.join(folder, filename), "r", encoding="utf-8", errors="ignore") as file:
                text = file.read()
                emails.append((text, label))
    return emails

In [8]:
spam_emails = load_emails(spam_folder, label=1)  
ham_emails = load_emails(ham_folder, label=0)   

In [13]:
df = pd.DataFrame(all_emails, columns=["text", "label"])
df.to_csv("dataset/emails.csv", index=False, escapechar='\\', quotechar='"')
print("CSV file created successfully!")

CSV file created successfully!


In [4]:
df = pd.read_csv("dataset/emails.csv")

### now we need to clean our messy dataset

In [5]:
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\raofb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\raofb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\raofb\AppData\Roaming\nltk_data...


True

In [8]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\raofb\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [6]:
stop_words = set(stopwords.words('english'))
def clean_text(text):
    # Lowercase
    text = text.lower()
    
    # Remove HTML
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Replace URLs and emails
    text = re.sub(r'http\S+', '<URL>', text)
    text = re.sub(r'\S+@\S+', '<EMAIL>', text)
    
    # Remove special characters (retain $ for monetary values)
    text = re.sub(r'[^\w\s$]', '', text)
    
    # Replace numbers (except after $)
    text = re.sub(r'\b\d+\b', '<NUM>', text)
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]

    return ' '.join(tokens)

### now we apply our nltk function to dataset

In [9]:
df = pd.read_csv('Dataset/emails.csv')
df['cleaned_text'] = df['text'].apply(clean_text)

In [10]:
df.to_csv('cleaned_emails.csv', index=False)

In [28]:
read = pd.read_csv("cleaned_emails.csv")
read.head(10)

,text,label,cleaned_text
0,Subject: fw : this is the solution i mentioned...,1,subject fw solution mentioned lsc oo thank ema...
1,"Subject: advs\ngreetings ,\ni am benedicta lin...",1,subject advs greeting benedicta lindiwe hendri...
2,Subject: whats new in summer ? bawled\ncarolyn...,1,subject whats new summer bawled carolyn regret...
3,"Subject: \nh $ ello\ndea 54 r home owner ,\nwe...",1,subject h $ ello dea < NUM > r home owner beet...
4,Subject: : ) ) you can not save the world by q...,1,subject save world quitting smoking save self ...
5,Subject: need software ? click here .\ntop qua...,1,subject need software click top quality softwa...
6,Subject: spend too much on your phone bill ? 2...,1,subject spend much phone bill < NUM > crystal ...
7,"Subject: slotting order confirmation may 18 , ...",1,subject slotting order confirmation may < NUM ...
8,Subject: we shiip to ur country for mircosoft ...,1,subject shiip ur country mircosoft adobe norto...
9,"Subject: urgent business proposal ,\nmrs . reg...",1,subject urgent business proposal mr regina ros...


In [30]:
df = pd.read_csv('cleaned_emails.csv')

# Drop the 'text' column
df = df.drop(columns=['text'])

# Rename 'cleaned_text' to 'text'
df = df.rename(columns={'cleaned_text': 'text'})

# Save the updated DataFrame to a new CSV file (or overwrite the existing one)
df.to_csv('Cleaned_data.csv', index=False)

### 2nd cleaning ---------------------------

In [38]:
from langdetect import detect, LangDetectException
from nltk.tokenize import word_tokenize, sent_tokenize

In [39]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    try:
        # Detect and remove non-English text
        if detect(text) != 'en':
            return ''
    except LangDetectException:
        return ''
    
    # Lowercase
    text = text.lower()
    
    # Remove HTML
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Enhanced URL/email replacement
    text = re.sub(r'\b(?:https?://|www\.)\S+', '<URL>', text)
    text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w{2,}\b', '<EMAIL>', text)
    
    # Date pattern replacement (MM/DD/YYYY, DD-MM-YYYY, Month names, etc.)
    text = re.sub(r'\b(\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b|(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*\s+\d{1,2},?\s+\d{4})', '<DATE>', text)
    
    # Time pattern replacement
    text = re.sub(r'\b\d{1,2}:\d{2}\s*(?:am|pm)?\b', '<TIME>', text)
    
    # Handle monetary values before general number replacement
    money_values = re.findall(r'\$\d+[\d,.]*', text)
    text = re.sub(r'\$\d+[\d,.]*', '<MONEY>', text)
    
    # Replace other numbers
    text = re.sub(r'\b\d+\b', '<NUM>', text)
    
    # Split and recombine long sentences
    sentences = sent_tokenize(text)
    processed_sentences = []
    for sent in sentences:
        # Limit sentence length to 15 words
        tokens = word_tokenize(sent)[:15]
        processed_sentences.append(' '.join(tokens))
    text = ' '.join(processed_sentences)
    
    # Remove special characters (keep $ for money handling)
    text = re.sub(r'[^\w\s<>]', '', text)
    
    # Tokenization and lemmatization
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return ' '.join(tokens)

In [40]:
df = pd.read_csv('Cleaned_data.csv')
df['cleaned_text'] = df['text'].apply(clean_text)


In [41]:
df.to_csv('cleaned_data1.csv', index=False)

#### we save our final data into our directories 

In [6]:
df = pd.read_csv('Final_data.csv')

# Drop the 'text' column
df = df.drop(columns=['text'])

# Rename 'cleaned_text' to 'text'
df = df.rename(columns={'cleaned_text': 'text'})

# Save the updated DataFrame to a new CSV file (or overwrite the existing one)
df.to_csv('test.csv', index=False)

### now we split our data into training and testing

In [8]:
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv("updated_dataset.csv")
print(df.head())  # we check the first row
print(df["category"].value_counts())  # Check classes

  category                                               text
0     spam  subject fw solution mentioned lsc oo thank ema...
1     spam  subject advs greeting benedicta lindiwe hendri...
2     spam  subject whats new summer bawled carolyn regret...
3     spam  subject h $ ello dea < NUM > r home owner beet...
4     spam  subject save world quitting smoking save self ...
category
spam    5996
ham     5861
Name: count, dtype: int64


In [9]:
X = df["text"]  # Input text
y = df["category"]  # Labels (0 or 1)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,           ## 20% of data for testing
    random_state=42,         ## Seed for reproducibility
    stratify=y               ## Preserve spam/ham ratio in both splits
)

In [1]:
import tensorflow as tf

### SAVE OUR TRAIN AND TEST DATA

In [11]:
train_df = pd.DataFrame({"text": X_train, "category": y_train})
test_df = pd.DataFrame({"text": X_test, "category": y_test})

# Save to CSV files
train_df.to_csv("Train and Test/train.csv", index=False)
test_df.to_csv("Train and Test/test.csv", index=False)

print(f"Training samples: {len(train_df)}")
print(f"Testing samples: {len(test_df)}")

Training samples: 9485
Testing samples: 2372


In [1]:
import pandas as pd

# Load your CSV dataset
df = pd.read_csv('Final_data.csv')

# Rename the 'label' column to 'category'
df.rename(columns={'label': 'category'}, inplace=True)

# Replace 0 with 'ham' and 1 with 'spam' in the 'category' column
df['category'] = df['category'].replace({0: 'ham', 1: 'spam'})

# Save the updated DataFrame back to a CSV (optional)
df.to_csv('updated_dataset.csv', index=False)

# Display the updated DataFrame
print(df.head())

  category                                               text
0     spam  subject fw solution mentioned lsc oo thank ema...
1     spam  subject advs greeting benedicta lindiwe hendri...
2     spam  subject whats new summer bawled carolyn regret...
3     spam  subject h $ ello dea < NUM > r home owner beet...
4     spam  subject save world quitting smoking save self ...


### Check our split data values

In [13]:
print("Train class distribution:\n", train_df["category"].value_counts())
print("Test class distribution:\n", test_df["category"].value_counts())

Train class distribution:
 category
spam    4796
ham     4689
Name: count, dtype: int64
Test class distribution:
 category
spam    1200
ham     1172
Name: count, dtype: int64


In [15]:
df_train = pd.read_csv("Train and Test/train.csv")
df_train.head(10)

,text,category
0,subject need outstanding logo working company ...,spam
1,subject weijun decided interview guess mean ba...,ham
2,subject investment partnership proposal dear s...,spam
3,subject tage resume submittal need time extern...,ham
4,subject trading ag prod vince wanted let know ...,ham
5,subject utf < NUM > q best dvd collect utf < N...,spam
6,subject portcalc methodology keith power emrs ...,ham
7,subject ibuyit approver ah wonderful new syste...,ham
8,subject osman sezgen steve kromer requesting o...,ham
9,subject read email change life dear homeowner ...,spam


In [18]:
df_ham = df_train[df_train['category']=='ham']
df_ham.shape

(4689, 2)

In [19]:
df_spam = df_train[df_train['category']=='spam']
df_spam.shape

(4796, 2)

In [22]:
df_balanced = pd.concat([df_ham, df_spam])
df_balanced.shape

(9485, 2)

In [26]:
df_balanced.tail(10)

,text,category
9456,subject get thru july newsletter get thru news...,spam
9458,subject cheap med nowrpmsqp every ph armaceuti...,spam
9459,subject trusted source prescription medication...,spam
9460,subject would hate reject chance reduce expend...,spam
9469,subject backplanes < NUM > vicodin whim cocked...,spam
9473,subject love girl idn wan na see girl naked pc...,spam
9476,subject clalis $ < NUM > < NUM > discount drug...,spam
9479,subject shoot spe rm drink increase cum volume...,spam
9483,subject george williams george williams abidja...,spam
9484,subject utf < NUM > q election result r utf < ...,spam


In [29]:
df_balanced['spam'] = df_balanced['category'].apply(lambda x: 1 if x=="spam" else 0)
df_balanced.sample(10)

,text,category,spam
2123,subject reminder cera executive roundtable hou...,ham,0
3480,subject fw tw docket rp < NUM > < NUM > < NUM ...,ham,0
1113,subject < NUM > sm unique logo business lack v...,spam,1
365,subject satellite dish click info stop getting...,spam,1
1920,subject truly amazed hi new product offer c _ ...,spam,1
2371,subject chess board widow living < NUM > hi,spam,1
8423,subject enron stanford program stinson monday ...,ham,0
7240,subject change lifestyle < NUM > k month bait ...,spam,1
7885,subject cera monthly summary july < NUM > cera...,ham,0
7869,subject save money buying ftware enough money ...,spam,1


### -----------------------------------------------Part-2------------------------------------------------------------------------

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import keras_nlp

In [2]:
preprocess_path = "/Users/techgarage/Projects/Data Engineering/Final Project/bert-tensorflow2-en-uncased-preprocess-v3"
encoder_path = "/Users/techgarage/Projects/Data Engineering/Final Project/bert-tensorflow2-en-uncased-l-12-h-768-a-12-v4"

In [3]:
bert_preprocess = hub.KerasLayer(preprocess_path)
bert_encoder = hub.KerasLayer(encoder_path, trainable=True) 

In [22]:
# Example input text
text = ["This is a sample sentence.", "BERT is powerful!"]

# Preprocess the text
preprocessed_text = bert_preprocess(text)

# Encode the text
bert_output = bert_encoder(preprocessed_text)

# bert_output contains the contextualized embeddings
print(bert_output)

{'default': <tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.9430887 , -0.57238555, -0.8907942 , ..., -0.8635234 ,
        -0.7363553 ,  0.93383116],
       [-0.8458787 , -0.3137438 , -0.5593949 , ..., -0.3633124 ,
        -0.6121973 ,  0.8722634 ]], dtype=float32)>, 'pooled_output': <tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.9430887 , -0.57238555, -0.8907942 , ..., -0.8635234 ,
        -0.7363553 ,  0.93383116],
       [-0.8458787 , -0.3137438 , -0.5593949 , ..., -0.3633124 ,
        -0.6121973 ,  0.8722634 ]], dtype=float32)>, 'sequence_output': <tf.Tensor: shape=(2, 128, 768), dtype=float32, numpy=
array([[[-0.19839856, -0.20800582, -0.1962688 , ..., -0.47272685,
          0.0864562 ,  0.7098722 ],
        [-0.5383126 , -0.7158387 , -0.28930873, ..., -0.72038037,
          0.5794679 ,  0.39614454],
        [-0.14156267, -0.7373614 ,  0.37026387, ..., -0.37185436,
          0.07691361,  0.9704544 ],
        ...,
        [ 0.06476687, -0.4518682 ,  0.2109

In [4]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435169 , -0.51327276, -0.8884573 , ..., -0.7474886 ,
        -0.75314736,  0.91964495],
       [-0.87208366, -0.5054398 , -0.9444669 , ..., -0.85847515,
        -0.71745366,  0.88082975]], dtype=float32)>

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
sl=get_sentence_embeding([
    "apple",
    "mango",
    "orange",
    "Bill gates",
    "tim cook",
    "jeff bezos"
])

In [7]:
cosine_similarity([sl[0]],[sl[1]])

array([[0.9614038]], dtype=float32)

In [8]:
cosine_similarity([sl[2]],[sl[3]])

array([[0.9564287]], dtype=float32)

In [9]:
cosine_similarity([sl[0]],[sl[4]])


array([[0.987975]], dtype=float32)

In [10]:
class BertPreprocessEncode(tf.keras.layers.Layer):
    def __init__(self, preprocess_path, encoder_path, **kwargs):
        super(BertPreprocessEncode, self).__init__(**kwargs)
        self.preprocess = hub.KerasLayer(preprocess_path)
        self.encoder = hub.KerasLayer(encoder_path)

    def call(self, inputs):
        preprocessed = self.preprocess(inputs)
        encoded = self.encoder(preprocessed)
        return encoded['pooled_output']

# Define the input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

# Use the custom preprocessing and encoding layer
bert_output = BertPreprocessEncode(preprocess_path, encoder_path)(text_input)

# Add a dropout layer and a dense layer for classification
dropout = tf.keras.layers.Dropout(0.1, name="dropout")(bert_output)
output = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(dropout)

# Build the model
model = tf.keras.Model(inputs=[text_input], outputs=[output])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Print the model summary
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text (InputLayer)               │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bert_preprocess_encode          │ (None, 768)            │             0 │
│ (BertPreprocessEncode)          │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │           769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769 (3.00 KB)

 Trainable params: 769 (3.00 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en_uncased",trainable=True)
encoder_inputs = preprocessor(text_input)
encoder = keras_nlp.models.BertBackbone.from_preset("bert_base_en_uncased")
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [11]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='test')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

ValueError: Exception encountered when calling layer 'keras_layer' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=None, name=test>
  • training=None

In [40]:
import numpy as np

# Example training data (spam detection)
train_texts = [
    "Win a free iPhone now!!!",  # Spam (1)
    "Meeting at 3 PM tomorrow"   # Ham (0)
]
train_labels = np.array([1, 0])  # Use numpy arrays for labels

# Convert to a TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_texts, train_labels))
train_dataset = train_dataset.batch(32)  # Adjust batch size as needed

# Train the model
model.fit(train_dataset, epochs=3)

Epoch 1/3


/Users/techgarage/anaconda3/envs/DE/lib/python3.9/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['text']
Received: inputs=Tensor(shape=(None,))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 1.0744
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - accuracy: 0.5000 - loss: 0.7670
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - accuracy: 0.5000 - loss: 0.7240


In [39]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the preprocessing and encoder layers from local directories
preprocess_path = "/Users/techgarage/Projects/Data Engineering/Final Project/bert-tensorflow2-en-uncased-preprocess-v3"
encoder_path = "/Users/techgarage/Projects/Data Engineering/Final Project/bert-tensorflow2-en-uncased-l-12-h-768-a-12-v4"

# Load the BERT layers
bert_preprocess = hub.KerasLayer(preprocess_path)
bert_encoder = hub.KerasLayer(encoder_path)

# Define the model architecture
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
encoded_output = bert_encoder(preprocessed_text)
pooled_output = encoded_output['pooled_output']

# Add classification head
dropout = tf.keras.layers.Dropout(0.1, name="dropout")(pooled_output)
output = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(dropout)

# Build the model
model = tf.keras.Model(inputs=[text_input], outputs=[output])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

ValueError: Exception encountered when calling layer 'keras_layer_4' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_4' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=False, name=text>
  • training=None

In [53]:
import os
os.environ['TF_USE_LEGACY_KERAS']='1'